In [1]:
!nvidia-smi

Mon May 16 07:36:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!pip install transformers==4.18.0
!pip install datasets

     |████████████████████████████████| 4.0 MB 5.0 MB/s 
     |████████████████████████████████| 6.6 MB 54.7 MB/s 
     |████████████████████████████████| 84 kB 4.1 MB/s 
     |████████████████████████████████| 596 kB 68.6 MB/s 
     |████████████████████████████████| 880 kB 65.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=ff24ebd7297d7bdf98145c3974fef3a995a5b780096fb7f1730c5cb3066110d1
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 342 kB 5.0 MB/s 
     |████████████████████████████████| 212 kB 70.1 MB/s 
     |████████████████████████████████| 136 kB 65.0 MB/s 
     |████████████████████████████████| 1.1 MB 52.4 MB/s 
     |██████████████████████████████

In [24]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


## Knowledge Distillation for Question Answering

__Teacher__: `XLM-Roberta-Large`

__Student__: `Multilingual-MiniLM-L12-H384`

In [25]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


## Models

In [26]:
from transformers import (AutoModelForQuestionAnswering, AutoTokenizer,
                          Trainer, TrainingArguments)

import collections

In [27]:
batch_size = 16
pretraining_language = 'ar'

In [28]:
teacher = AutoModelForQuestionAnswering.from_pretrained("bhavikardeshna/xlm-roberta-base-chinese")
teacher_tokenizer = AutoTokenizer.from_pretrained("bhavikardeshna/xlm-roberta-base-chinese")


student_ckpt = f"subhasisj/{pretraining_language}-TAPT-MLM-MiniLM"
student_tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

loading configuration file https://huggingface.co/bhavikardeshna/xlm-roberta-base-chinese/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ef1153f75e58369d8d05c6d2066ae6ac70f88be7dbba92736eff3b4f5a2954a9.5f82b93a3e9e239bcaeef19084205a2f1bfc42586ec02bd8c10cd3e99eaa0d61
Model config XLMRobertaConfig {
  "_name_or_path": "bhavikardeshna/xlm-roberta-base-chinese",
  "architectures": [
    "XLMRobertaForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "language": "english",
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "name": "XLMRoberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_typ

In [29]:
# Tokenize text and infer from context using QA model
question = "How much music can this hold?"
context = """An MP3 is about 1 MB/minute, so about 6000 hours depending on \
file size."""
inputs = teacher_tokenizer(question, context, return_tensors="pt")
print(teacher_tokenizer.decode(inputs["input_ids"][0]))

<s> How much music can this hold?</s></s> An MP3 is about 1 MB/minute, so about 6000 hours depending on file size.</s>


In [30]:
with torch.no_grad():
    outputs = teacher(**inputs)
print(outputs)

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-0.7309, -3.5360, -5.0358, -4.3353, -5.5025, -4.4434, -3.9800, -7.6311,
         -6.6683, -6.1203,  1.7861,  1.4719, -2.5172, -2.0107,  3.1427,  5.0705,
          2.2360, -2.1821, -1.8110, -3.7949, -4.1963, -1.8203,  2.9620,  3.9244,
         -0.9758, -1.9594, -4.4129, -3.9839, -0.0980, -2.9271, -3.8002, -3.8469]]), end_logits=tensor([[-1.2647, -5.4138, -3.8071, -3.2306, -5.8194, -5.0057, -4.3430, -6.4385,
         -6.3842, -6.1430, -2.3061, -1.9739,  0.7059, -3.5490, -1.3366,  1.5516,
          3.3999, -2.2032, -0.6195,  3.3264,  0.7030, -1.8500, -2.0669,  2.4836,
          3.8447, -3.4636, -1.2111, -3.0987, -1.5339,  0.1944,  2.1928, -3.0521]]), hidden_states=None, attentions=None)


## Data

In [31]:
import numpy as np
import pandas as pd
import json

def squad_json_to_dataframe_train(input_file_path, record_path = ['data','paragraphs','qas','answers'],
                           verbose = 1):
    """
    input_file_path: path to the squad json file.
    record_path: path to deepest level in json file default value is
    ['data','paragraphs','qas','answers']
    verbose: 0 to suppress it default is 1
    """
    if verbose:
        print("Reading the json file")    
    file = json.loads(open(input_file_path).read())
    if verbose:
        print("processing...")
    # parsing different level's in the json file
    js = pd.io.json.json_normalize(file , record_path )
    m = pd.io.json.json_normalize(file, record_path[:-1] )
    r = pd.io.json.json_normalize(file,record_path[:-2])
    
    #combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
    ndx  = np.repeat(m['id'].values,m['answers'].str.len())
    m['context'] = idx
    js['q_idx'] = ndx
    main = pd.concat([ m[['id','question','context']].set_index('id'),js.set_index('q_idx')],1,sort=False).reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape))
        print("Done")
    return main

In [32]:
# training data
language = pretraining_language
input_file_path = f'./drive/MyDrive/Colab Files/squad.translate.train.en-{language}.json'
record_path = ['data','paragraphs','qas','answers']
train = squad_json_to_dataframe_train(input_file_path=input_file_path,record_path=record_path,verbose=0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only


In [33]:
def get_answers(x):
    start = x[0]
    text = x[1]
    return {
        'answer_start': [start],
        'text': [text]
    }

train['answers'] = train[['answer_start', 'text']].apply(get_answers, axis=1)
pd.set_option('display.max_colwidth',None)
train.head(1)

,index,question,context,answer_start,text,c_id,answers
0,5706071252bb8914006897cd,أين تزود المهاجرين بالوقود؟,بالنسبة لبعض أنواع الخواضون ، يعتمد نجاح الهجرة على توفر بعض موارد الغذاء الرئيسية في نقاط التوقف على طول طريق الترحيل. هذا يعطي المهاجرين فرصة للتزود بالوقود من أجل المحطة التالية من الرحلة. بعض الأمثلة على مواقع التوقف المهمة هي Bay of Fundy و Delaware Bay.,86,نقاط التوقف على طول طريق الترحيل,0,"{'answer_start': [86], 'text': ['نقاط التوقف على طول طريق الترحيل']}"


In [34]:
# Validation data
input_file_path = f'./drive/MyDrive/Colab Files/squad.translate.dev.en-{language}.json'
record_path = ['data','paragraphs','qas','answers']
dev = squad_json_to_dataframe_train(input_file_path=input_file_path,record_path=record_path,verbose=0)
dev['answers'] = dev[['answer_start', 'text']].apply(get_answers, axis=1)
pd.set_option('display.max_colwidth',None)
dev.head(1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only


,index,question,context,answer_start,text,c_id,answers
0,5729da0faf94a219006aa678,ما نوع الأجور الناتجة عن الوظائف التي يكون فيها العرض منخفضًا ولكن الطلب مرتفعًا؟,إن الوظيفة التي يوجد فيها عدد كبير من العمال الراغبين في العمل لفترة طويلة من الوقت (ارتفاع العرض) يتنافسون على وظيفة تتطلبها قلة (انخفاض الطلب) ستؤدي إلى انخفاض الأجر عن هذه الوظيفة. وذلك لأن المنافسة بين العمال تدفع الأجور. مثال على ذلك سيكون وظائف مثل غسل الصحون أو خدمة العملاء. تميل المنافسة بين العمال إلى خفض الأجور بسبب الطبيعة المستهلكة للعامل فيما يتعلق بوظيفته. إن الوظيفة التي يوجد فيها عدد قليل من العمال القادرين أو الراغبين (نقص العرض) ، ولكن هناك حاجة كبيرة للوظائف (ارتفاع الطلب) ، ستؤدي إلى ارتفاع أجر لهذه الوظيفة. وذلك لأن التنافس بين أرباب العمل على الموظفين سيؤدي إلى رفع الأجور. تشمل الأمثلة على ذلك الوظائف التي تتطلب مهارات متطورة للغاية ، أو قدرات نادرة ، أو درجة عالية من المخاطرة. تميل المنافسة بين أرباب العمل إلى رفع الأجور بسبب طبيعة الوظيفة ، نظرًا لوجود نقص نسبي في عدد العمال في هذا المنصب المحدد. قد تحد المنظمات المهنية والعمال من عرض العمال مما يؤدي إلى زيادة الطلب وزيادة الدخول للأعضاء. قد يحصل الأعضاء أيضًا على أجور أعلى من خلال المفاوضة الجماعية أو التأثير السياسي أو الفساد.,509,ارتفاع,0,"{'answer_start': [509], 'text': ['ارتفاع']}"


### Create Datasets

In [35]:
from datasets import Dataset,DatasetDict


tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(dev)


squad = DatasetDict()

squad['train'] = tds
squad['validation'] = vds

In [36]:
squad

DatasetDict({
    train: Dataset({
        features: ['index', 'question', 'context', 'answer_start', 'text', 'c_id', 'answers'],
        num_rows: 86787
    })
    validation: Dataset({
        features: ['index', 'question', 'context', 'answer_start', 'text', 'c_id', 'answers'],
        num_rows: 34448
    })
})

In [37]:
pad_on_right = student_tokenizer.padding_side == "right"
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = student_tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(student_tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

def postprocess_qa_predictions(
    examples, features, raw_predictions, n_best_size=20, max_answer_length=30
):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(
        f"Post-processing {len(examples)} example predictions split into {len(features)} features."
    )

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None  # Only used if squad_v2 is True.
        valid_answers = []

        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(
                tokenizer.cls_token_id
            )
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[
                -1 : -n_best_size - 1 : -1
            ].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                        or offset_mapping[start_index] == []
                        or offset_mapping[end_index] == []
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char:end_char],
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[
                0
            ]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}

        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        # if not squad_v2:
        # predictions[example["id"]] = best_answer["text"]
        # else:
        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer

    return predictions

In [38]:
squad = squad.map(prepare_train_features,batched=True,remove_columns=squad["train"].column_names)

  0%|          | 0/87 [00:00<?, ?ba/s]

  0%|          | 0/35 [00:00<?, ?ba/s]

## Training

In [39]:
import torch.nn as nn
import torch.nn.functional as F

class DistillationTrainingArguments(TrainingArguments):
    def __init__(self, *args, alpha=0.5, temperature=2.0, **kwargs):
        super().__init__(*args, **kwargs)
        self.alpha = alpha
        self.temperature = temperature

class DistillationTrainer(Trainer):
    def __init__(self, *args, teacher_model=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher_model = teacher_model

    def compute_loss(self, model, inputs,return_outputs=False):
        outputs_stu = model(**inputs)
        # Extract cross-entropy loss and logits from student
        loss_ce = outputs_stu.loss
        # Extract logits from teacher
        with torch.no_grad():
            outputs_tea = self.teacher_model(**inputs)
            start_logits_tea = outputs_tea.start_logits
            end_logits_tea = outputs_tea.end_logits
        # Soften probabilities and compute distillation loss
        loss_fct = nn.KLDivLoss(reduction="batchmean")
        
        loss_start = loss_fct(
                    nn.functional.log_softmax(outputs_stu.start_logits / self.args.temperature, dim=-1),
                    nn.functional.softmax(start_logits_tea / self.args.temperature, dim=-1),
                ) * (self.args.temperature**2)
        loss_end = loss_fct(
            nn.functional.log_softmax(outputs_stu.end_logits / self.args.temperature, dim=-1),
            nn.functional.softmax(end_logits_tea / self.args.temperature, dim=-1),
        ) * (self.args.temperature**2)
        loss_kd = (loss_start + loss_end) / 2.0

        loss = self.args.alpha * loss_ce + (1 - self.args.alpha) * loss_kd

        return loss

In [40]:
from transformers import default_data_collator


from tqdm.auto import tqdm


def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)
    
def student_init():
    return AutoModelForQuestionAnswering.from_pretrained(
        f"subhasisj/{pretraining_language}-TAPT-MLM-MiniLM"
    ).to(device)


student_training_args = DistillationTrainingArguments(
    output_dir = f"./{pretraining_language}-kd-XLM-minilmv2-{batch_size}",
    # evaluation_strategy="epoch",
    save_strategy = "epoch",
    learning_rate = 3e-5,
    num_train_epochs=5,
    per_device_train_batch_size=batch_size,
    # per_device_eval_batch_size=batch_size,
    temperature = 2.0,
    alpha=0.5,
    warmup_ratio = 0.1,
    fp16=True,
    gradient_accumulation_steps = 8,
    weight_decay=0.01,
    push_to_hub=True,
    save_total_limit=1,
    # load_best_model_at_end=True
)

teacher.to(device)


data_collator = default_data_collator

distil_trainer = DistillationTrainer(
    model_init=student_init,
    teacher_model=teacher,
    args=student_training_args,
    train_dataset=squad["train"],
    # eval_dataset=squad["validation"],
    data_collator=data_collator,
    tokenizer=student_tokenizer,
    # compute_metrics=compute_metrics,
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/subhasisj/ar-TAPT-MLM-MiniLM/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/f1e964ae98e6ee43ab8ed956aa7f1a59a9d86d02afa1ecfe1b753de417eaff93.e10bd0b72b4099c977ef126c1372e211950889239eb7513aba618c575d204753
Model config BertConfig {
  "_name_or_path": "subhasisj/ar-TAPT-MLM-MiniLM",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "nu

In [ ]:
distil_trainer.train()

loading configuration file https://huggingface.co/subhasisj/ar-TAPT-MLM-MiniLM/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/f1e964ae98e6ee43ab8ed956aa7f1a59a9d86d02afa1ecfe1b753de417eaff93.e10bd0b72b4099c977ef126c1372e211950889239eb7513aba618c575d204753
Model config BertConfig {
  "_name_or_path": "subhasisj/ar-TAPT-MLM-MiniLM",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "XLMRobertaTokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 250037
}

load

Step,Training Loss
500,3.830400


Saving model checkpoint to ./ar-kd-XLM-minilmv2-16/checkpoint-704
Configuration saved in ./ar-kd-XLM-minilmv2-16/checkpoint-704/config.json
Model weights saved in ./ar-kd-XLM-minilmv2-16/checkpoint-704/pytorch_model.bin
tokenizer config file saved in ./ar-kd-XLM-minilmv2-16/checkpoint-704/tokenizer_config.json
Special tokens file saved in ./ar-kd-XLM-minilmv2-16/checkpoint-704/special_tokens_map.json
tokenizer config file saved in ./ar-kd-XLM-minilmv2-16/tokenizer_config.json
Special tokens file saved in ./ar-kd-XLM-minilmv2-16/special_tokens_map.json
Adding files tracked by Git LFS: ['tokenizer.json']. This may take a bit of time if the files are large.


In [ ]:
distil_trainer.push_to_hub("Training Completed")